In [2]:
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal


# 
import numpy as np
import os
from tqdm import trange

# 
from scipy.io import loadmat
import scipy

# 
from Convert import Convert

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# # data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/centres/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres.npy')
# # print (data.shape)
# data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_fixed.npy')
# print (data.shape)

In [3]:
# 
fnames = [
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-6_0day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-7_daytime_0_300sec_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-8_0day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-9_1day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-10_daytime_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-11_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-12_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-13_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-14_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-15_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-16_day_5mins_compressedTalmo_fixed.npy',
]

# 
for fname in fnames:
    #
    convert = Convert(fname)

    # 
    # convert.sexes = ['F','M','M','M']
    convert.sexes = [0,1,1,1]
    convert.fps = 25
    convert.start = 0 #int(0*60*convert.fps)
    convert.end = None #None #int(1*60*convert.fps)

    # start = 0
    # end = None

    convert.dtype = 'double'
    convert.animal_ids = np.arange(4)
    convert.scale = 4
    convert.make_movie = True
    #convert.axes_scale = 4

    # 
    convert.extension = '.avi'
    convert.convert_npy_to_jaaba()

    #break

    

  1%|          | 40/7500 [00:00<00:19, 392.21it/s]

 full features track data:  (7500, 4, 14, 2)


  1%|          | 65/7500 [00:00<00:11, 637.36it/s]

 full features track data:  (7500, 4, 14, 2)


  1%|          | 45/7500 [00:00<00:16, 448.37it/s]

...

  0%|          | 35/7500 [00:00<00:21, 347.93it/s]

 full features track data:  (7500, 4, 14, 2)


  2%|▏         | 113/7500 [00:00<00:06, 1119.32it/s]

 full features track data:  (7500, 4, 14, 2)


100%|██████████| 7500/7500 [00:27<00:00, 272.77it/s]


In [13]:
from tqdm import trange
clrs = ['red','blue','green','cyan']

dirs = np.sort(glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/'+
                "*"))

imgs=[]
imgs1 = []
for dir_ in dirs:
    fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres/scores_run.mat'
    data =loadmat(fname_mat)
    
    # get length
    temp = data['allScores'][0][0][0][0][0].squeeze()
    length = temp.shape[0]

    # get starts/ends
    t0s = data['allScores'][0][0][5].squeeze()
    #print (t0s.shape)
    t1s = data['allScores'][0][0][6].squeeze()
    
    img = np.zeros((4,length),'float32')
    img1 = np.zeros((4,length),'float32')
    for k in trange(t0s.shape[0]):
    #for k in [0]:
        starts = t0s[k].squeeze()
        ends = t1s[k].squeeze()
        #print (k,starts.shape)
        for p in range(starts.shape[0]):
            img[k,starts[p]:ends[p]]+=k+1
            img1[k,starts[p]:ends[p]]+=1
            
    idx = np.where(img==0)
    img[idx]=np.nan
    imgs.append(img)
    imgs1.append(img1)
    
imgs=np.hstack(imgs)
imgs1=np.hstack(imgs1)


100%|██████████| 4/4 [00:00<00:00, 15.94it/s]


In [18]:
matplotlib.use('Agg')

fig = plt.figure(figsize=(100,5))
ax=plt.subplot(2,1,1)


plt.xticks([])
print (imgs.shape)
names = ['female','male','pup1','pup2']
clrs = ['red','blue','magenta','green']
plt.yticks(np.arange(4),names)
from matplotlib import colors
cmap = colors.ListedColormap(clrs)

plt.imshow(imgs,aspect='auto',interpolation='none',
          cmap=cmap)

ax=plt.subplot(2,1,2)
sums = np.nansum(imgs1, axis= 0)
print (sums.shape)

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    #y_smooth = np.convolve(y, box, mode='same')
    y_smooth = np.convolve(y,np.ones(box_pts,dtype=int),'same')
    
    return y_smooth

window = 60
t = np.arange(imgs.shape[1])/25./window
img2 = np.zeros((4,imgs.shape[1]))
for k in range(4):
    temp = imgs1[k]
    temp = smooth(temp,25*window)    
    #img2[k]=temp
    #temp = smooth(temp,25)    
    
    plt.plot(t,temp,clrs[k])

#plt.imshow(img2,aspect='auto',interpolation='none')#    cmap=cmap)    
plt.xticks(np.arange(0,600,10))
plt.xlim(t[0],t[-1])
plt.xlabel("Time (minutes)")
plt.suptitle("Run behaviors for all animals ("+str(window)+ "sec smoothing window)", fontsize=20)
plt.savefig('/home/cat/fig.png')
#plt.show()



(4, 899880)
(899880,)


In [100]:
temp = data['allScores'][0][0][0][0][k].squeeze()
print(temp.shape)


(89988,)


In [63]:
t0s = data['allScores'][0][0][5].squeeze()
print (t0s.shape)
t1s = data['allScores'][0][0][6].squeeze()

(4,)


In [167]:
fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx_all_21300_21500.mat'
data = loadmat(fname_mat)

In [168]:
print (data['trx'].shape)
print (data['trx'][0][0][1].shape)

data['trx'][0][0][1][0][0] = 44
data['trx'][0][0][1][0] = np.array(data['trx'][0][0][1][0],dtype)

print (data['trx'][0][0][1][0][0])

(1, 4)
(1, 200)
44.0


In [ ]:
  
    
from numpy.core.records import fromarrays
from scipy.io import savemat

names = [
 'x',
 'y',
 'theta',
 'a',
 'b',
 'nframes',
 'firstframe',
 'endframe',
 'off',
 'id',
 'x_mm',
 'y_mm',
 'theta_mm',
 'a_mm',
 'b_mm',
 'sex',
 'dt',
 'fps',
 'timestamps']

data = [
 x,
 y,
 theta,
 a,
 b,
 nframes,
 firstframe,
 endframe,
 off,
 id_,
 x_mm,
 y_mm,
 theta_mm,
 a_mm,
 b_mm,
 sex,
 dt,
 fps,
 timestamps
]


            
#myrec = fromarrays([[1, 10], [2, 20]], names=['field1', 'field2'])
#savemat('p.mat', {'myrec': myrec})

myrec = fromarrays(data, 
                   names=names)
#savemat('p.mat', {'myrec': myrec})


savemat("/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx.mat", 
        {'myrec':myrec})
    

In [11]:
fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/*.npy')

for fname in fnames:
    data = np.load(fname)
    
    data2 = []
    for f in range(0,56,14):
        temp = data[:,f:f+14]
        temp = np.nanmedian(temp,axis=1)
        data2.append(temp)
        
    data2=np.array(data2)
    np.save(fname[:-4]+'_centres.npy',data2)
    print (data2.shape)

(4, 89988, 2)
(4, 89985, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89987, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89989, 2)
(4, 89976, 2)
(4, 89988, 2)


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import scale

# load iris dataset
iris = datasets.load_iris()

X = scale(iris.data)
y = iris.target

# apply PCA
pca = decomposition.PCA(n_components=4)
X = pca.fit_transform(X)

loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', "PC2", "PC3"], index=iris.feature_names)
print (loadings)


                        PC1       PC2       PC2       PC3
sepal length (cm)  0.521066  0.377418 -0.719566 -0.261286
sepal width (cm)  -0.269347  0.923296  0.244382  0.123510
petal length (cm)  0.580413  0.024492  0.142126  0.801449
petal width (cm)   0.564857  0.066942  0.634273 -0.523597


In [18]:
d1 = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_fixed.npy')
print (d1.shape)

(89989, 4, 14, 2)


In [19]:
# angles = np.zeros((track.tracks.shape[0], 
#                    track.tracks.shape[1], 
#                    'float32'))

angles = np.zeros((d1.shape[0], 
                   d1.shape[1]), 
                   'float32')

deg_scale = 180/3.1415926

for k in trange(10000):
    for a in range(d1.shape[1]):
        x = d1[k,a,:,0]
        y = d1[k,a,:,1]
        idx = np.where(np.isnan(x)==False)[0]
        if idx.shape[0]>0:
            x=x[idx]
            y=y[idx]
            m,b = np.polyfit(x, y, 1)
            #print ("slope :", m, "  intercept: ", b)
            angle = np.arctan(m)*deg_scale
            angles[k,a] = angle
            #print ("agnel: ", angle)      
            

100%|██████████| 10000/10000 [00:06<00:00, 1578.38it/s]


In [ ]:
plt.scatter()

In [87]:
# remove outliars
def reject_outliers(data, m = 20):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    idx = np.where(s<m)[0]
    
    return idx
    #return data[s<m]
        
deg_scale = 180/3.1415926

k_start= 1060
a = 0
ctr=1
for k in range(k_start,k_start+10,1):
    #ax = plt.gca()
    ax =plt.subplot(2,5,ctr)
    x = d1[k,a,5:10,0]
    y = d1[k,a,5:10,1]
    idx = np.where(np.isnan(x)==False)[0]
    if idx.shape[0]>0:
        x=x[idx]
        y=y[idx]
        m,b = np.polyfit(y, x, 1)
        angle = np.arctan(m)*deg_scale #+90
        if x[0]<x[-1]:
            angle+=180
        print ("angle: ", angle) 
        
    plt.title(str(round(angle,1))+" "+str(k))

    locs = np.vstack((x,y)).T
    plt.scatter(locs[:,0],
                locs[:,1],
                c='blue',
                label='original')
    y_vals = np.array(ax.get_ylim())
    x_vals = b + m * y_vals
    plt.plot(x_vals, y_vals, '--')
    plt.xlim(0,1280)
    plt.ylim(1024,0)

    # rotate the thang
    if False:
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array(((c, -s), (s, c)))
        print (locs.shape)
        #locs = R*locs.T
        locs = locs@R



        # Reject outliers that are substantially outside of data
        x = reject_outliers(locs[:,0])
        y = reject_outliers(locs[:,1])

        major = np.max(x)-np.min(x)
        minor = np.max(y)-np.min(y)
        print ("major: ", major, "  minor: ", minor)

        # 
        plt.scatter(locs[:,0], 
                    locs[:,1],
                    c='red',
                   label='rotated')
    
    ctr+=1
plt.legend()
plt.show()

angle:  155.36937322044065
angle:  156.3964094037172
angle:  158.820461411969
angle:  162.98178696290174
angle:  163.0214896740499
angle:  163.20703257794614
angle:  163.1095933974912
angle:  164.4740648932155
angle:  165.45458477460073
angle:  169.03968922275195
